## Applying a simple Tensorflow model to MNIST data (with optimisation)

In [1]:
#importing the required packages
import tensorflow as tf
from tensorflow import keras

In [2]:
#importing the data
data = keras.datasets.mnist.load_data()

In [3]:
data

((array([[[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         ...,
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          ...,
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0]],
  
         [[0, 0, 0, ..., 0, 0, 0],
          [0, 0, 0, ..., 0, 0, 0

In [4]:
#splitting in test and train
(Xtrain_val,ytrain_val), (Xtest,ytest) = keras.datasets.mnist.load_data()

In [5]:
Xtrain_val.shape

(60000, 28, 28)

In [6]:
Xtrain_val.max()

255

In [7]:
Xval, Xtrain = Xtrain_val[:5000]/255.0, Xtrain_val[5000:]/255.0
yval, ytrain = ytrain_val[:5000], ytrain_val[5000:]
Xtest = Xtest/255.0

In [8]:
#building the classifier - as the optimal learning rate has to be found, using randomised search and building it 
#as a sklearn regressor
def build_model(n_hidden=2, n_neurons=100, learning_rate=1e-3, input_shape=[28,28]):
    model = keras.models.Sequential()
    model.add(keras.layers.Flatten(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation='relu'))
    model.add(keras.layers.Dense(10, activation='softmax'))
    optimizer = keras.optimizers.SGD(lr=learning_rate)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [9]:
#save checkpoints, use early stopping, plot learning curves in tensorboard
#writing the callbacks
checkpoint_cb = keras.callbacks.ModelCheckpoint('mnist_mlp_model.h5')
early_stop_cb = keras.callbacks.EarlyStopping(patience=10)

In [10]:
#setting up for tensorboard
import os
import time
root_logdir = os.path.join(os.curdir, 'logs')
run_logdir = os.path.join(root_logdir, time.strftime('run_%Y_%m_%d-%H_%M_%S'))

In [11]:
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)
#creating the classifier
keras_clf = keras.wrappers.scikit_learn.KerasClassifier(build_model)

In [12]:
#using randomised search
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
lr_range = np.arange(1e-3, 5, 0.001)
param_dist = {'learning_rate': lr_range}
rnd_search = RandomizedSearchCV(keras_clf, param_dist, n_iter=100, cv=3, verbose=2)
rnd_search.fit(Xtrain, ytrain, epochs=100, validation_data=(Xval, yval), 
             callbacks=[tensorboard_cb, checkpoint_cb, early_stop_cb])

Fitting 3 folds for each of 100 candidates, totalling 300 fits
Epoch 1/100
1146/1146 [==============================] - 9s 7ms/step - loss: 2.1127 - accuracy: 0.2255 - val_loss: 2.3008 - val_accuracy: 0.1008
Epoch 2/100
1146/1146 [==============================] - 7s 6ms/step - loss: 2.2559 - accuracy: 0.1270 - val_loss: 2.3433 - val_accuracy: 0.1790
Epoch 3/100
1146/1146 [==============================] - 7s 6ms/step - loss: 2.1734 - accuracy: 0.1606 - val_loss: 2.3128 - val_accuracy: 0.0986
Epoch 4/100
1146/1146 [==============================] - 7s 6ms/step - loss: 2.3104 - accuracy: 0.1052 - val_loss: 2.3076 - val_accuracy: 0.1002
Epoch 5/100
1146/1146 [==============================] - 7s 6ms/step - loss: 2.3108 - accuracy: 0.1023 - val_loss: 2.3049 - val_accuracy: 0.1126
Epoch 6/100
1146/1146 [==============================] - 7s 6ms/step - loss: 2.3113 - accuracy: 0.0991 - val_loss: 2.3111 - val_accuracy: 0.0868
Epoch 7/100
1146/1146 [==============================] - 6s 5ms/ste

Epoch 12/100
1146/1146 [==============================] - 7s 6ms/step - loss: 2.3428 - accuracy: 0.1001 - val_loss: 2.3350 - val_accuracy: 0.0986
Epoch 13/100
1146/1146 [==============================] - 5s 5ms/step - loss: 2.3397 - accuracy: 0.1035 - val_loss: 2.3112 - val_accuracy: 0.0924
Epoch 14/100
1146/1146 [==============================] - 6s 5ms/step - loss: 2.3443 - accuracy: 0.0993 - val_loss: 2.3387 - val_accuracy: 0.0924
Epoch 15/100
1146/1146 [==============================] - 6s 5ms/step - loss: 2.3459 - accuracy: 0.0997 - val_loss: 2.3939 - val_accuracy: 0.1126
Epoch 16/100
1146/1146 [==============================] - 6s 5ms/step - loss: 2.3460 - accuracy: 0.1000 - val_loss: 2.3351 - val_accuracy: 0.0924
Epoch 17/100
1146/1146 [==============================] - 6s 5ms/step - loss: 2.3474 - accuracy: 0.0972 - val_loss: 2.3392 - val_accuracy: 0.0924
Epoch 18/100
1146/1146 [==============================] - 6s 5ms/step - loss: 2.3451 - accuracy: 0.1033 - val_loss: 2.3560 -

Epoch 15/100
1146/1146 [==============================] - 6s 5ms/step - loss: 2.3209 - accuracy: 0.1018 - val_loss: 2.3287 - val_accuracy: 0.1070
Epoch 16/100
1146/1146 [==============================] - 7s 6ms/step - loss: 2.3201 - accuracy: 0.1013 - val_loss: 2.3299 - val_accuracy: 0.1126
Epoch 17/100
1146/1146 [==============================] - 6s 6ms/step - loss: 2.3212 - accuracy: 0.0989 - val_loss: 2.3338 - val_accuracy: 0.1126
Epoch 18/100
573/573 [==============================] - 3s 4ms/step - loss: 2.3320 - accuracy: 0.1142
[CV] END ................................learning_rate=2.452; total time= 1.9min
Epoch 1/100
1146/1146 [==============================] - 9s 7ms/step - loss: 70.7933 - accuracy: 0.1020 - val_loss: 2.3273 - val_accuracy: 0.1100
Epoch 2/100
1146/1146 [==============================] - 7s 6ms/step - loss: 2.3208 - accuracy: 0.1015 - val_loss: 2.3433 - val_accuracy: 0.0868
Epoch 3/100
1146/1146 [==============================] - 7s 6ms/step - loss: 2.3208 - ac

Epoch 10/100
1146/1146 [==============================] - 3s 3ms/step - loss: 2.3191 - accuracy: 0.1058 - val_loss: 2.3168 - val_accuracy: 0.1126
Epoch 11/100
1146/1146 [==============================] - 5s 4ms/step - loss: 2.3199 - accuracy: 0.1015 - val_loss: 2.3306 - val_accuracy: 0.0976
Epoch 12/100
1146/1146 [==============================] - 5s 4ms/step - loss: 2.3205 - accuracy: 0.1019 - val_loss: 2.3295 - val_accuracy: 0.0986
Epoch 13/100
1146/1146 [==============================] - 5s 4ms/step - loss: 2.3208 - accuracy: 0.1016 - val_loss: 2.3270 - val_accuracy: 0.0986
Epoch 14/100
573/573 [==============================] - 3s 5ms/step - loss: 2.3292 - accuracy: 0.1142
[CV] END ................................learning_rate=2.374; total time= 1.2min
Epoch 1/100
1146/1146 [==============================] - 8s 6ms/step - loss: 1732.0182 - accuracy: 0.1027 - val_loss: 2.3143 - val_accuracy: 0.1002
Epoch 2/100
1146/1146 [==============================] - 5s 5ms/step - loss: 2.3192 -

Epoch 11/100
1146/1146 [==============================] - 2s 2ms/step - loss: 2.3167 - accuracy: 0.1049 - val_loss: 2.3265 - val_accuracy: 0.1100
Epoch 12/100
1146/1146 [==============================] - 2s 2ms/step - loss: 2.3152 - accuracy: 0.1042 - val_loss: 2.3222 - val_accuracy: 0.1070
Epoch 13/100
573/573 [==============================] - 1s 1ms/step - loss: 2.3164 - accuracy: 0.1144
[CV] END ................................learning_rate=1.944; total time=  51.5s
Epoch 1/100
1146/1146 [==============================] - 3s 2ms/step - loss: 51.4089 - accuracy: 0.1034 - val_loss: 2.3161 - val_accuracy: 0.0958
Epoch 2/100
1146/1146 [==============================] - 2s 2ms/step - loss: 2.3168 - accuracy: 0.1029 - val_loss: 2.3204 - val_accuracy: 0.1126
Epoch 3/100
1146/1146 [==============================] - 2s 1ms/step - loss: 2.3169 - accuracy: 0.1023 - val_loss: 2.3222 - val_accuracy: 0.1126
Epoch 4/100
1146/1146 [==============================] - 2s 1ms/step - loss: 2.3164 - acc

...

Epoch 12/100
1146/1146 [==============================] - 2s 1ms/step - loss: 2.3112 - accuracy: 0.1058 - val_loss: 2.3167 - val_accuracy: 0.1126
Epoch 13/100
1146/1146 [==============================] - 2s 1ms/step - loss: 2.3120 - accuracy: 0.1053 - val_loss: 2.3154 - val_accuracy: 0.1126
Epoch 14/100
1146/1146 [==============================] - 2s 1ms/step - loss: 2.3117 - accuracy: 0.1063 - val_loss: 2.3127 - val_accuracy: 0.1002
Epoch 15/100
1146/1146 [==============================] - 2s 1ms/step - loss: 2.3122 - accuracy: 0.1036 - val_loss: 2.3158 - val_accuracy: 0.0958
Epoch 16/100
1146/1146 [==============================] - 2s 2ms/step - loss: 2.3120 - accuracy: 0.1031 - val_loss: 2.3093 - val_accuracy: 0.1126
Epoch 17/100
1146/1146 [==============================] - 2s 1ms/step - loss: 2.3124 - accuracy: 0.1023 - val_loss: 2.3113 - val_accuracy: 0.1126
Epoch 18/100
1146/1146 [==============================] - 2s 2ms/step - loss: 2.3110 - accuracy: 0.1040 - val_loss: 2.3063 -

Epoch 12/100
1146/1146 [==============================] - 2s 1ms/step - loss: 2.3183 - accuracy: 0.0995 - val_loss: 2.3107 - val_accuracy: 0.1100
Epoch 13/100
1146/1146 [==============================] - 2s 1ms/step - loss: 2.3164 - accuracy: 0.1014 - val_loss: 2.3076 - val_accuracy: 0.1126
Epoch 14/100
1146/1146 [==============================] - 2s 1ms/step - loss: 2.3162 - accuracy: 0.1055 - val_loss: 2.3030 - val_accuracy: 0.1126
Epoch 15/100
1146/1146 [==============================] - 2s 1ms/step - loss: 2.3169 - accuracy: 0.1009 - val_loss: 2.3126 - val_accuracy: 0.1100
Epoch 16/100
1146/1146 [==============================] - 2s 1ms/step - loss: 2.3176 - accuracy: 0.1033 - val_loss: 2.3171 - val_accuracy: 0.0986
Epoch 17/100
1146/1146 [==============================] - 2s 1ms/step - loss: 2.3189 - accuracy: 0.1004 - val_loss: 2.3146 - val_accuracy: 0.1100
Epoch 18/100
1146/1146 [==============================] - 2s 1ms/step - loss: 2.3180 - accuracy: 0.1045 - val_loss: 2.3135 -

Epoch 1/100
1146/1146 [==============================] - 3s 2ms/step - loss: 2798.8117 - accuracy: 0.1035 - val_loss: 2.3228 - val_accuracy: 0.0990
Epoch 2/100
1146/1146 [==============================] - 2s 2ms/step - loss: 2.3359 - accuracy: 0.0977 - val_loss: 2.3690 - val_accuracy: 0.1002
Epoch 3/100
1146/1146 [==============================] - 2s 1ms/step - loss: 2.3343 - accuracy: 0.0993 - val_loss: 2.3272 - val_accuracy: 0.0924
Epoch 4/100
1146/1146 [==============================] - 2s 2ms/step - loss: 2.3308 - accuracy: 0.1080 - val_loss: 2.3403 - val_accuracy: 0.1126
Epoch 5/100
1146/1146 [==============================] - 2s 1ms/step - loss: 2.3329 - accuracy: 0.0959 - val_loss: 2.3619 - val_accuracy: 0.1126
Epoch 6/100
1146/1146 [==============================] - 2s 2ms/step - loss: 2.3316 - accuracy: 0.1031 - val_loss: 2.3404 - val_accuracy: 0.0958
Epoch 7/100
1146/1146 [==============================] - 2s 1ms/step - loss: 2.3342 - accuracy: 0.1032 - val_loss: 2.3512 - val

1146/1146 [==============================] - 2s 1ms/step - loss: 2.3428 - accuracy: 0.1049 - val_loss: 2.3490 - val_accuracy: 0.0986
Epoch 7/100
1146/1146 [==============================] - 2s 2ms/step - loss: 2.3471 - accuracy: 0.1013 - val_loss: 2.3361 - val_accuracy: 0.0924
Epoch 8/100
1146/1146 [==============================] - 2s 1ms/step - loss: 2.3460 - accuracy: 0.1000 - val_loss: 2.3522 - val_accuracy: 0.1070
Epoch 9/100
1146/1146 [==============================] - 2s 1ms/step - loss: 2.3437 - accuracy: 0.1013 - val_loss: 2.3342 - val_accuracy: 0.1002
Epoch 10/100
1146/1146 [==============================] - 2s 1ms/step - loss: 2.3424 - accuracy: 0.1035 - val_loss: 2.3629 - val_accuracy: 0.1126
Epoch 11/100
573/573 [==============================] - 1s 1ms/step - loss: 2.3370 - accuracy: 0.0981
[CV] END ...................learning_rate=4.6930000000000005; total time=  20.5s
Epoch 1/100
1146/1146 [==============================] - 3s 2ms/step - loss: 9932.5637 - accuracy: 0.10

RandomizedSearchCV(cv=3,
                   estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x000001E4149D9A00>,
                   n_iter=100,
                   param_distributions={'learning_rate': array([1.000e-03, 2.000e-03, 3.000e-03, ..., 4.997e+00, 4.998e+00,
       4.999e+00])},
                   verbose=2)

In [13]:
rnd_search.best_params_

{'learning_rate': 0.14}

In [14]:
rnd_search.best_score_

0.9757635990778605

In [15]:
model = rnd_search.best_estimator_.model

In [21]:
ypred_1 = np.argmax(model.predict(Xtest), axis=-1)

In [22]:
from sklearn.metrics import accuracy_score
accuracy_score(ytest, ypred_1)

0.9815